In [1]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [ ]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

In [ ]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

In [ ]:
df = pd.concat([df_train , df_test],axis= 0)
print df.shape
df.head()

In [ ]:
#df_train.isnull().sum()
df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [ ]:
#df_train.isnull().sum()
df['Consumer-complaint-summary'] = df['Consumer-complaint-summary'].apply(preprocess_data)

In [ ]:
import re
df['Consumer-complaint-summary'] = df['Consumer-complaint-summary'].map(lambda x : re.sub("x+","",x))

In [ ]:
df["merge"] = df["Consumer-complaint-summary"].map(str)+ " " + df["Complaint-reason"]

In [1]:
import pandas as pd
df = pd.read_csv('/home/prashant/preprocess_brain_test_train.csv')

In [5]:
len(df.loc[df['tag'] == 'test']['Complaint-reason'].value_counts())

137

In [ ]:
def get_vector(model,all_data):
    vector_sen = [
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        for w in words:
            try:
                
                get_word_vec = model[w]
            except:
                get_word_vec =np.zeros(300)
                pass
            single_sen_vec.append(get_word_vec)
        v = np.array(single_sen_vec).mean(axis=0)
        vector_sen.append(v)
    return vector_sen

In [ ]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

model = loadGloveModel('/home/prashant/Downloads/kagsa/data/glove.840B.300d.txt')
#vocab = model.keys()

In [3]:
text = list(df['merge'])
vectorizer = TfidfVectorizer(min_df=15,max_df=100)
# # tokenize and build vocab
vectorizer.fit(text)
#vector = get_vector(model,list(df['merge']))

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=100, max_features=None, min_df=15,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [ ]:
# # encode document
# vector = vectorizer.transform(text)
# # summarize encoded vector
# print(vector.shape)


In [ ]:
vec_df = pd.DataFrame(vector)

In [ ]:
vec_df['tag'] = list(df['tag'])
vec_df['Complaint-Status'] = list(df['Complaint-Status'])
vec_df['Transaction-Type'] = list(df['Transaction-Type'])
vec_df['Complaint-ID'] = list(df['Complaint-ID'])

In [ ]:
complete_df_one_hot = vec_df

train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)


In [ ]:
y_train = (train_data['Complaint-Status'])
x_train = train_data.drop(['Complaint-Status','Complaint-ID'],axis =1)

In [ ]:
print y_train.shape
y_train_one_hot = pd.get_dummies(y_train,prefix=['Complaint-Status'])
print y_train_one_hot.shape

In [ ]:
#df.to_csv('/home/prashant/preprocess_brain.csv',index=False)

In [ ]:
print x_train.shape
print y_train.shape
x_train = pd.get_dummies(x_train,prefix=['Transaction-Type'])

In [ ]:
test_data = pd.get_dummies(test_data,prefix=['Transaction-Type'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,random_state = 0)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

# Train the model on training data
rm = rf.fit(X_train, y_train_internal)
pred_internal = rm.predict(X_test)



In [ ]:
import sklearn
sklearn.metrics.f1_score(pred_internal,y_test_internal,average='weighted')

In [ ]:
id_ = list(test_data['Complaint-ID'])


In [ ]:
print train_data.shape
test_data.shape
test_data.drop(['Complaint-Status','Complaint-ID'],inplace=True,axis=1)

In [ ]:
y_pred = rm.predict(test_data)

In [ ]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = y_pred

In [ ]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/baseline_approx_rf_2.csv',index=None)

In [ ]:
df_test.head()